<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/Copy_of_WEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEAT Algorithm
## Test Statistic

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# returns s(w, A, B) for all w in W (passed as argument). Shape: n_words (in W) x 1
def swAB(W, A, B):
  #Calculate cosine-similarity between W and A, W and B
  #print("W: ", W.shape, " A: ", A.shape, " B: ", B.shape)
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  #print('WA shape: ', WA.shape)
  #Take mean along columns
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)
  
  #print('sWAB shape: ', WAmean.shape)
  
  return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
  #Convert the set of words to matrix
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  # Find X U Y
  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w.lower() in embd:
      XuYmat.append(embd[w.lower()])
  XuYmat = np.array(XuYmat)
#   print('X U Y Shape: ', XuYmat.shape)
#   print('X Shape: ', Xmat.shape)
#   print('Y Shape: ', Ymat.shape)
#   print('A Shape: ', Amat.shape)
#   print('B Shape: ', Bmat.shape)
  
  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
  
  return d

## P-Value

In [0]:
import random
from itertools import combinations, filterfalse

def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample):
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
      
  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
    Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
    
  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  #print("All: ", test_stats_over_permutation)
  #print("Unpertrubed: ", unperturbed)
  return is_over.sum() / is_over.size

## Test on sample input

In [0]:
X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"] #Instruments
Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"] #Weapons
A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] #Pleasant
B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] #Unpleasant

#Load word embeddings
#load gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''


glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))

The glove embedding has been loaded!
WEAT d =  1.5495627
Unpertrubed:  2.2905553244054317
WEAT p =  0.0


## WEAT with conceptor debiased embeddings

### Compute the conceptor matrix for all words and gender specific words.

In [0]:
#Compute the conceptor matrix
def process_cn_matrix(subspace, alpha = 2):
  #print("starting...")
  #x = orig_embd.vectors
  #print(subspace.shape)
  
  #Calculate the correlation matrix
  R = subspace.dot(subspace.T)/(subspace.shape[1])
  #print("R calculated")
  
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(subspace.shape[0])))
  #print("C calculated")
  
  #Calculate the negation of the conceptor matrix
  negC = np.eye(subspace.shape[0]) - C
  #print("negC calculated")
  
  return negC

def apply_conceptor(x, C):
  #Post-process the vocab matrix
  newX = (C @ x).T
  print(newX.shape)
  return newX

Load embeddings of all words from the ref. wordlist from a specific embedding

In [0]:
#Arguments - embd: The word embeddings in the form of a dict || wikiWordsPath: List of words to be considered
def load_all_vectors(embd, wikiWordsPath):
  all_words_index = {}
  all_words_mat = []
  with open(wikiWordsPath, "r+") as f_in:
    ind = 0
    for line in f_in:
      word = line.split(' ')[0]
      if word in embd:
        all_words_index[word] = ind
        all_words_mat.append(embd[word])
        ind = ind+1
        
  return all_words_index, all_words_mat

def load_subspace_vectors(embd, subspace_words):
  subspace_embd_mat = []
  ind = 0
  for word in subspace_words:
    if word in embd:
      subspace_embd_mat.append(embd[word])
      ind = ind+1
      
  return subspace_embd_mat

In [66]:
all_words_index, all_words_mat = load_all_vectors(glove, wikiWordsPath)

aaa = list(all_words_index.keys())
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = ['T' for w in X if w.lower() in aaa]
print(a)
print(np.array(a).shape)

['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil', 'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith', 'Sarah']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
(18,)


Conceptor all words and store it in a dictonary

### Load all word lists - Subspace

In [109]:
# General word list
!wget https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
!git clone https://github.com/PrincetonML/SIF
    
# Gender word lists
!git clone https://github.com/uclanlp/gn_glove
!git clone https://github.com/uclanlp/corefBias
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
    
# our code for debiasing -- also includes word lists    
!git clone https://github.com/jsedoc/ConceptorDebias

--2019-03-03 09:38:35--  https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23741395 (23M) [text/plain]
Saving to: ‘enwiki-20150602-words-frequency.txt.2’

enwiki-20150602-wor 100%[===================>]  22.64M  47.3MB/s    in 0.5s    

2019-03-03 09:38:36 (47.3 MB/s) - ‘enwiki-20150602-words-frequency.txt.2’ saved [23741395/23741395]

fatal: destination path 'SIF' already exists and is not an empty directory.
fatal: destination path 'gn_glove' already exists and is not an empty directory.
fatal: destination path 'corefBias' already exists and is not an empty directory.
--2019-03-03 09:38:41--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/n

In [110]:
!ls

ConceptorDebias			       female.txt    male.txt.1
corefBias			       female.txt.1  male.txt.2
enwiki-20150602-words-frequency.txt    female.txt.2  SIF
enwiki-20150602-words-frequency.txt.1  gn_glove
enwiki-20150602-words-frequency.txt.2  male.txt


In [0]:
from load_word_lists import *

gender_list_pronouns = WEATLists.W_7_Male_terms + WEATLists.W_7_Female_terms + WEATLists.W_8_Male_terms + WEATLists.W_8_Female_terms
gender_list_pronouns = list(set(gender_list_pronouns))

gender_list_extended = male_vino_extra + female_vino_extra + male_gnGlove + female_gnGlove
gender_list_extended = list(set(gender_list_extended))

gender_list_propernouns = male_cmu + female_cmu
gender_list_propernouns = list(set(gender_list_propernouns))

gender_list_all = gender_list_pronouns + gender_list_extended + gender_list_propernouns
gender_list_all = list(set(gender_list_all))

race_list = WEATLists.W_3_Unused_full_list_European_American_names + WEATLists.W_3_European_American_names + WEATLists.W_3_Unused_full_list_African_American_names + WEATLists.W_3_African_American_names + WEATLists.W_4_Unused_full_list_European_American_names + WEATLists.W_4_European_American_names + WEATLists.W_4_Unused_full_list_African_American_names + WEATLists.W_4_African_American_names + WEATLists.W_5_Unused_full_list_European_American_names + WEATLists.W_5_European_American_names + WEATLists.W_5_Unused_full_list_African_American_names + WEATLists.W_5_African_American_names 
race_list = list(set(race_list))

### Load different embeddings

Glove

In [74]:
!ls


ConceptorDebias  examples.desktop  nltk_data


In [112]:
#Load word embeddings
#download gensim formatted Full Glove embeddings
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /home/saketk/content/gensim_glove.840B.300d.txt.bin
2.65GB [00:24, 110MB/s]
The glove embedding has been loaded!


In [0]:
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = [glove[w] for w in X if w.lower() in glove]
print(np.array(a).shape)
glove['Brad']
#glove['brad']

Word2vec

In [113]:
#load gensim formatted Full Word2vec embeddings
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
!gunzip GoogleNews-vectors-negative300.bin.gz
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The word2vec embedding has been loaded!')

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /home/saketk/content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:18, 89.5MB/s]
The word2vec embedding has been loaded!


Fasttext

In [114]:
!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

import gensim
from gensim.models.keyedvectors import KeyedVectors  

resourceFile = ''
fasttext = KeyedVectors.load_word2vec_format(resourceFile + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

Downloading...
From: https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh
To: /home/saketk/content/fasttext.bin
2.42GB [00:21, 113MB/s]
The fasttext embedding has been loaded!


Elmo

In [0]:
!gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU
  
import pickle
data = pickle.load(open("elmo_embeddings_emma_brown.pkl", "rb"))

def pick_embeddings(corpus, sent_embs):
    X = []
    labels = {}
    sents = []
    ind = 0
    for i, s in enumerate(corpus):
        for j, w in enumerate(s):
            X.append(sent_embs[i][j])
            if w.lower() in labels:
              labels[w.lower()].append(ind)
            else:
              labels[w.lower()] = [ind]
            sents.append(s)
            ind = ind + 1
    return (X, labels, sents)
  
def get_word_list(path):
    word_list = []
    with open(path, "r+") as f_in:
      for line in f_in:
        word = line.split(' ')[0]
        word_list.append(word.lower())

    return word_list

def load_subspace_vectors_contextual(all_mat, all_index, subspace_list):
    subspace_mat = []
    for w in subspace_list:
      if w.lower() in all_index:
        for i in all_index[w.lower()]:
          #print(type(i))
          subspace_mat.append(all_mat[i])
    #subspace_mat = [all_mat[i,:] for i in all_index[w.lower()] for w in subspace_list if w.lower() in all_index]
    print("Subspace: ", np.array(subspace_mat).shape)
    return subspace_mat

In [0]:
import nltk
from nltk.corpus import brown

#nltk.download('brown')

brown_corpus = brown.sents()
elmo = data['brown_embs']


In [0]:
print(np.array(elmo_brown_mat).shape)
print(len(list(elmo_brown_index.keys())))
print(len(wiki_words))

In [113]:
a = [[1,2,3], [4,5,6]]
np.mean(a, axis = 0)

array([2.5, 3.5, 4.5])

Bert

elmo_gender_list_all.pkl       elmo_gender_list_propernouns.pkl
elmo_gender_list_extended.pkl  elmo_race_list.pkl
elmo_gender_list_pronouns.pkl


In [166]:
np.matrix(data2['type_embedings'])

matrix([[4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        [4096],
        

###Post-process (CN) all embeddings using a particular subspace

In [131]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['glove', 'word2vec', 'fasttext','elmo']
# all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']
all_subspace = ['without_conceptor', 'race_list']

# career = WEATLists.W_6_Career
# family = WEATLists.W_6_Family
# male = WEATLists.W_6_Male_names
# female = WEATLists.W_6_Female_names

white = WEATLists.W_5_Unused_full_list_European_American_names
black = WEATLists.W_5_Unused_full_list_African_American_names
pleasant = WEATLists.W_5_Pleasant
unpleasant = WEATLists.W_5_Unpleasant

# science = WEATLists.W_8_Science
# arts = WEATLists.W_8_Arts
# male = WEATLists.W_8_Male_terms
# female = WEATLists.W_8_Female_terms

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    #wiki_words = get_word_list('SIF/auxiliary_data/enwiki_vocab_min200.txt')
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_conceptor' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_conceptor':
      #CN all word embeddings using the respective subspace
      if subspace == 'gender_list_and':
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat1 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T, alpha = 8)

          subspace_words_mat2 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T, alpha = 3)

          subspace_words_mat3 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T, alpha = 10)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("All mat CN: ", np.array(all_words_cn).shape)
        else:
          subspace_words_mat1 = load_subspace_vectors(curr_embd, gender_list_pronouns)
          cn1 = process_cn_matrix(np.array(subspace_words_mat1).T)

          subspace_words_mat2 = load_subspace_vectors(curr_embd, gender_list_extended)
          cn2 = process_cn_matrix(np.array(subspace_words_mat2).T)

          subspace_words_mat3 = load_subspace_vectors(curr_embd, gender_list_propernouns)
          cn3 = process_cn_matrix(np.array(subspace_words_mat3).T)

          cn = AND(cn1, AND(cn2, cn3))
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      else:
        #Load all embeddings of the subspace as a matrix
        if embd == 'elmo' or embd == 'bert':
          subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T, alpha = 6)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
          print("Subspace mat: ", np.array(subspace_words_mat).shape)
        else:
          subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
          cn = process_cn_matrix(np.array(subspace_words_mat).T)
          all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
#     print("LAST: ", np.array(all_words["a"]).shape)
    if subspace == 'without_conceptor':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("Without conceptor")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("With conceptor: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

All CN:  (128607, 300)
Without conceptor
WEAT d =  1.3542663
WEAT p =  0.0
(128607, 300)
All CN:  (128607, 300)
With conceptor:  glove race_list
WEAT d =  0.6888005504919007
WEAT p =  0.007
All CN:  (76078, 300)
Without conceptor
WEAT d =  -0.27154148
WEAT p =  0.265
(76078, 300)
All CN:  (76078, 300)
With conceptor:  word2vec race_list
WEAT d =  -0.5498752519776564
WEAT p =  0.724
All CN:  (119127, 300)
Without conceptor
WEAT d =  0.41335136
WEAT p =  0.038
(119127, 300)
All CN:  (119127, 300)
With conceptor:  fasttext race_list
WEAT d =  -0.2710636522184819
WEAT p =  0.573
All mat:  (1161192, 768)
Number of words:  49815
All CN:  (1161192, 768)
Without conceptor
WEAT d =  1.3741117
WEAT p =  0.0
Subspace:  (751, 768)
(1161192, 768)
Subspace mat:  (751, 768)
All CN:  (1161192, 768)
With conceptor:  elmo race_list
WEAT d =  -0.45138745986044126
WEAT p =  0.2


,Embedding,Subspace,Effect Size,p-value
0,glove,without_conceptor,1.3542663,0.0
1,glove,race_list,0.6888005504919007,0.007
2,word2vec,without_conceptor,-0.27154148,0.265
3,word2vec,race_list,-0.5498752519776564,0.724
4,fasttext,without_conceptor,0.41335136,0.038
5,fasttext,race_list,-0.2710636522184819,0.573
6,elmo,without_conceptor,1.3741117,0.0
7,elmo,race_list,-0.45138745986044126,0.2


In [127]:
if w.lower() not in all_words_index:
  print("T")
#load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)

T


### Calculate WEAT scores

In [0]:
science = WEATLists.W_8_Science
arts = WEATLists.W_8_Arts
male = WEATLists.W_8_Male_terms
female = WEATLists.W_8_Female_terms

#WITHOUT CONCEPTOR
print("Without conceptor")
print('WEAT d = ', weat_effect_size(science, arts, male, female, curr_embd))
print('WEAT p = ', weat_p_value(science, arts, male, female, curr_embd, 1000))

#WITH CONCEPTOR
print("With conceptor")
print('WEAT d = ', weat_effect_size(science, arts, male, female, all_words))
print('WEAT p = ', weat_p_value(science, arts, male, female, all_words, 1000))

Without conceptor
WEAT d =  1.2780356
Unpertrubed:  0.3456038013100624
WEAT p =  0.003
With conceptor
WEAT d =  0.681611936758009
Unpertrubed:  0.10700200023922478
WEAT p =  0.103


## WEAT algorithm from GITHUB gist
REF: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5

In [0]:
class WEATTest(object):
    """
    Perform WEAT (Word Embedding Association Test) bias tests on a language model.
    Follows from Caliskan et al 2017 (10.1126/science.aal4230).
    """
    
    instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"]
    weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"]
    flowers = ["aster", "clover", "hyacinth", "marigold", "poppy", "azalea", "crocus", "iris", "orchid", "rose", "blue-bell", "daffodil", "lilac", "pansy", "tulip", "buttercup", "daisy", "lily", "peony", "violet", "carnation", "gladiola", "magnolia", "petunia", "zinnia"]
    insects = ["ant", "caterpillar", "flea", "locust", "spider", "bedbug", "centipede", "fly", "maggot", "tarantula", "bee", "cockroach", "gnat", "mosquito", "termite", "beetle", "cricket", "hornet", "moth", "wasp", "blackfly",
    "dragonfly", "horsefly", "roach", "weevil"]
    pleasant = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"]
    unpleasant = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"]
    math = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
    arts = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
    male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
    female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
    science = ["science", "technology", "physics", "chemistry", "Einstein", "NASA", "experiment", "astronomy"]
    arts_2 = ["poetry", "art", "Shakespeare", "dance", "literature", "novel", "symphony", "drama"]
    male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
    female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
    mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]
    physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
    temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]
    permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]
    
    def __init__(self, model):
        """Setup a Word Embedding Association Test for a given spaCy language model.
        
        EXAMPLE:
            >>> nlp = spacy.load('en_core_web_md')
            >>> test = WEATTest(nlp)
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
        """
        self.model = model

    @staticmethod
    def word_association_with_attribute(self, w, A, B):
        return np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(a).reshape(1,-1)) for a in A]) - np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(b).reshape(1,-1)) for b in B])

    @staticmethod
    def differential_assoication(self, X, Y, A, B):
        return np.sum([self.word_association_with_attribute(self, x, A, B) for x in X]) - np.sum([self.word_association_with_attribute(self, y, A, B) for y in Y])

    @staticmethod
    def weat_effect_size(self, X, Y, A, B):
        return (
            np.mean([self.word_association_with_attribute(self, x, A, B) for x in X]) -
            np.mean([self.word_association_with_attribute(self, y, A, B) for y in Y])
        ) / np.std([self.word_association_with_attribute(self, w, A, B) for w in X + Y])

    @staticmethod
    def random_permutation(self, iterable, r=None):
        pool = tuple(iterable)
        r = len(pool) if r is None else r
        return tuple(random.sample(pool, r))

    @staticmethod
    def weat_p_value(self, X, Y, A, B, sample):
        size_of_permutation = min(len(X), len(Y))
        X_Y = X + Y
        observed_test_stats_over_permutations = []

        if not sample:
            permutations = combinations(X_Y, size_of_permutation)
        else:
            permutations = [self.random_permutation(self, X_Y, size_of_permutation) for s in range(sample)]
        print(np.array(X_Y).shape)
        for Xi in permutations:
            Yi = filterfalse(lambda w:w in Xi, X_Y)
            observed_test_stats_over_permutations.append(self.differential_assoication(self, Xi, Yi, A, B))

        unperturbed = self.differential_assoication(self, X, Y, A, B)
        is_over = np.array([o > unperturbed for o in observed_test_stats_over_permutations])
        return is_over.sum() / is_over.size

    @staticmethod
    def weat_stats(X, Y, A, B, self, sample_p=None):
        test_statistic = self.differential_assoication(self, X, Y, A, B)
        effect_size = self.weat_effect_size(self, X, Y, A, B)
        p = self.weat_p_value(self, X, Y, A, B, sample=sample_p)
        return test_statistic, effect_size, p

    def run_test(self, target_1, target_2, attributes_1, attributes_2, sample_p=None):
        """Run the WEAT test for differential association between two 
        sets of target words and two seats of attributes.
        
        EXAMPLE:
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
            >>> test.run_test(a, b, c, d, sample_p=1000) # use 1000 permutations for p-value calculation
            >>> test.run_test(a, b, c, d, sample_p=None) # use all possible permutations for p-value calculation
            
        RETURNS:
            (d, e, p). A tuple of floats, where d is the WEAT Test statistic, 
            e is the effect size, and p is the one-sided p-value measuring the
            (un)likeliness of the null hypothesis (which is that there is no
            difference in association between the two target word sets and
            the attributes).
            
            If e is large and p small, then differences in the model between 
            the attribute word sets match differences between the targets.
        """
        X = [list(self.model[w]) for w in target_1]
        Y = [list(self.model[w]) for w in target_2]
        A = [list(self.model[w]) for w in attributes_1]
        B = [list(self.model[w]) for w in attributes_2]
        print(X)
        return self.weat_stats(X, Y, A, B, self, sample_p)


## Code test

In [0]:
#nlp = spacy.load('glove')
test = WEATTest(glove)
test.run_test(WEATTest.instruments, WEATTest.weapons, WEATTest.pleasant, WEATTest.unpleasant, 1000)

The glove embedding has been loaded!
X Shape:  (3, 300)
Y Shape:  (3, 300)
A Shape:  (2, 300)
B Shape:  (2, 300)
X U Y Shape:  (6, 300)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (6, 2)
sWAB shape:  (6,)
WEAT d =  1.8613524


In [232]:
!ls

bert_W_10_Old_peoples_names.pkl
bert_W_10_Pleasant.pkl
bert_W_10_Unpleasant.pkl
bert_W_10_Young_peoples_names.pkl
bert_W_1_Flowers.pkl
bert_W_1_Insects.pkl
bert_W_1_Pleasant.pkl
bert_W_1_Unpleasant.pkl
bert_W_2_Instruments.pkl
bert_W_2_Pleasant.pkl
bert_W_2_Unpleasant.pkl
bert_W_2_Weapons.pkl
bert_W_3_African_American_names.pkl
bert_W_3_European_American_names.pkl
bert_W_3_Pleasant.pkl
bert_W_3_Unpleasant.pkl
bert_W_3_Unused_full_list_African_American_names.pkl
bert_W_3_Unused_full_list_European_American_names.pkl
bert_W_4_African_American_names.pkl
bert_W_4_European_American_names.pkl
bert_W_4_Pleasant.pkl
bert_W_4_Unpleasant.pkl
bert_W_4_Unused_full_list_African_American_names.pkl
bert_W_4_Unused_full_list_European_American_names.pkl
bert_W_5_African_American_names.pkl
bert_W_5_European_American_names.pkl
bert_W_5_Pleasant.pkl
bert_W_5_Unpleasant.pkl
bert_W_5_Unused_full_list_African_American_names.pkl
bert_W_5_Unused_full_list_European_American_names.pkl
bert_W_6_Career.pkl
bert_W_6

In [236]:
# import os

# all = {}
# for filename in os.listdir('/home/saketk/bert'):
#   all[filename] = pickle.load(open(filename, "rb"))
res = all['big_bert_gender_list_extended.pkl']['type_embedings']
w = []
for name in all:
  res = np.concatenate((res, all[name]['type_embedings']))
  w += all[name]['words']
print(res.shape)
print(len(set(w)))
w = [aaa.lower() for aaa in w]
cn_pronouns = all['big_bert_gender_list_pronouns.pkl']['GnegC']
cn_propernouns = all['big_bert_gender_list_propernouns.pkl']['GnegC']
cn_extended = all['big_bert_gender_list_extended.pkl']['GnegC']
cn_all = all['big_bert_gender_list_all.pkl']['GnegC']
cn_race = all['big_bert_race_list.pkl']['GnegC']

print(np.array(cn_all).shape)

(975, 4096)
493
(4096, 4096)


In [0]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['bert']
all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']
# all_subspace = ['without_conceptor', 'race_list']

all_words_mat = res
all_words_index = {}
print(len(set(w)))
for i,a in enumerate(w):
  all_words_index[a] = i
# all_words_index = {wr:i for i,wr in enumerate(w)}
print("All_words_mat: ", all_words_mat.shape)
print("Index: ", len(all_words_index.keys()))
career = WEATLists.W_8_Science
family = WEATLists.W_8_Arts
male = WEATLists.W_8_Male_terms
female = WEATLists.W_8_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant
embd = 'bert'
results = []
for subspace in all_subspace:
    
  if subspace != 'without_conceptor' and subspace != 'gender_list_and':
    subspace_words_list = eval(subspace)

  if subspace != 'without_conceptor':
    #CN all word embeddings using the respective subspace
    if subspace == 'gender_list_and':
      cn = AND(cn_pronouns, AND(cn_extended, cn_propernouns))
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("All mat CN: ", np.array(all_words_cn).shape)
    else:
      #Load all embeddings of the subspace as a matrix
      if subspace == 'gender_list_pronouns':
        cn = cn_pronouns
      elif subspace == 'gender_list_propernouns':
        cn = cn_propernouns
      elif subspace == 'gender_list_extended':
        cn = cn_extended
      elif subspace == 'gender_list_all':
        cn = cn_all
        
      print("CN shape: ", np.array(cn).shape)
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("Subspace mat: ", np.array(subspace_words_mat).shape)
      
  else:
    all_words_cn = all_words_mat
  all_words_cn = np.array(all_words_cn)
  print("All CN: ", all_words_cn.shape)
  #Store all conceptored words in a dictonary
  all_words = {}
  for word, index in all_words_index.items():
    all_words[word] = all_words_cn[index,:]
  print("D: ", len(all_words.keys()))
  if subspace == 'without_conceptor':
    #WITHOUT CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("Without conceptor")
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)
  else:
    #WITH CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("With conceptor: ", embd, subspace)
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)

  row = [embd, subspace, d_cn, p_cn]
  results.append(row)

  
  
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

493
All_words_mat:  (975, 4096)
Index:  493
All CN:  (975, 4096)
D:  493
Without conceptor
WEAT d =  -0.9136145
WEAT p =  0.884
CN shape:  (4096, 4096)
(975, 4096)
Subspace mat:  (751, 768)
All CN:  (975, 4096)
D:  493
With conceptor:  bert gender_list_pronouns
WEAT d =  -1.2300879460348944
WEAT p =  0.974
CN shape:  (4096, 4096)
(975, 4096)
Subspace mat:  (751, 768)
All CN:  (975, 4096)
D:  493
With conceptor:  bert gender_list_extended
WEAT d =  -1.1197445478394272
WEAT p =  0.994
CN shape:  (4096, 4096)
(975, 4096)
Subspace mat:  (751, 768)
All CN:  (975, 4096)
D:  493
With conceptor:  bert gender_list_propernouns
WEAT d =  -0.9280666738182892
WEAT p =  0.922
CN shape:  (4096, 4096)
(975, 4096)
Subspace mat:  (751, 768)
All CN:  (975, 4096)
D:  493
With conceptor:  bert gender_list_all
WEAT d =  -0.3839682787101548
WEAT p =  0.704


In [184]:
a = [[1,2,3,4],[3,2,1,4]]
b = [[1,2,3,4],[6,7,8,9],[1,4,3,2]]
d = [[33,44,55,66]]
c = np.concatenate((a,b,d))
print(c)

[[ 1  2  3  4]
 [ 3  2  1  4]
 [ 1  2  3  4]
 [ 6  7  8  9]
 [ 1  4  3  2]
 [33 44 55 66]]


# Hard Debiasing

In [118]:
!mkdir debiaswe_tutorial
%cd debiaswe_tutorial
!git clone https://github.com/tolga-b/debiaswe.git

/home/saketk/content/debiaswe_tutorial
Cloning into 'debiaswe'...
remote: Enumerating objects: 91, done.
remote: Total 91 (delta 0), reused 0 (delta 0), pack-reused 91
Unpacking objects: 100% (91/91), done.
Checking connectivity... done.


In [0]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np
from debiaswe import debiaswe
from debiaswe.debiaswe import we
import debiaswe as dwe
#import debiaswe.we as we
from debiaswe.debiaswe.we import WordEmbedding
from debiaswe.debiaswe.data import load_professions
from debiaswe.debiaswe import debias
from debiaswe.debiaswe.debias import debias

In [119]:
%cd debiaswe

/home/saketk/content/debiaswe_tutorial/debiaswe


In [30]:
#!gdown https://drive.google.com/uc?id=1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG
!unzip w2v_gnews_small.zip


Archive:  w2v_gnews_small.zip
  inflating: w2v_gnews_small.txt     


In [23]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f) #gender definitional words
print("definitional", defs)
defs_list = []
for pair in defs:
  defs_list.append(pair[0])
  defs_list.append(pair[1])

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f) 
print("Equalize pairs", equalize_pairs)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
Equalize pairs [['monastery', 'convent'], ['spokesman', 'spokeswoman'], ['Catholic_priest', 'nun'], ['Dad', 'Mom'], ['Men', 'Women'], ['councilman', 'councilwoman'], ['grandpa', 'grandma'], ['grandsons', 'granddaughters'], ['prostate_cancer', 'ovarian_cancer'], ['testosterone', 'estrogen'], ['uncle', 'aunt'], ['wives', 'husbands'], ['Father', 'Mother'], ['Grandpa', 'Grandma'], ['He', 'She'], ['boy', 'girl'], ['boys', 'girls'], ['brother', 'sister'], ['brothers', 'sisters'], ['businessman', 'businesswoman'], ['chairman', 'chairwoman'], ['colt', 'filly'], ['congressman', 'congresswoman'], ['dad', 'mom'], ['dads', 'moms'], ['dudes', 'gals'], ['ex_girlfriend', 'ex_boyfriend'], ['father', 'mother'], ['fatherhood', 'motherhood'], ['fathers', 'mothers'], ['fella', 'granny'], ['fraternity', 'soro

In [31]:
E = WordEmbedding('w2v_gnews_small.txt')

*** Reading data from w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine


In [35]:
E1 =  WordEmbedding('../GoogleNews-vectors-negative300.bin')

*** Reading data from ../GoogleNews-vectors-negative300.bin
(3000000, 300)
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus
3000000 words of dimension 300 : </s>, in, for, that, ..., Bim_Skala_Bim, Mezze_Cafe, pulverizes_boulders, snowcapped_Caucasus


## Mu et. al. Hard Debiasing

In [85]:
from sklearn.decomposition import PCA
def hard_debias(all_words, subspace):
  all_words = np.array(all_words)
  subspace = np.array(subspace)
  print(all_words.shape, " ", subspace.shape)
  pca = PCA(n_components = 1)
  pca.fit(subspace)
  pc1 = np.array(pca.components_)
  
  temp = (pc1.T @ (pc1 @ all_words.T)).T
  ret = all_words - temp
  
  return ret
  
  
subspace = [[1,2,3,4],[11,2,3,12]]

all_words = [[1,2,3,4], [5,6,7,8],[21,3,1,4],[11,2,3,12], [6,7,4,3]]
print(hard_debias(all_words, subspace))

(5, 4)   (2, 4)
[[-1.56097561  2.          3.          1.95121951]
 [-1.95121951  6.          7.          2.43902439]
 [ 6.24390244  3.          1.         -7.80487805]
 [-1.56097561  2.          3.          1.95121951]
 [ 0.87804878  7.          4.         -1.09756098]]


In [55]:
subspace = np.array([1,2,3,4])
subspace = subspace.T

aaa = np.array([2,4,1,3,10])
b = np.multiply(aaa, subspace)

ValueError: ignored

In [102]:
import numpy as np
import pandas as pd


resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['glove', 'word2vec', 'fasttext','elmo']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
# all_subspace = ['without_debiasing', 'race_list']

# science = WEATLists.W_8_Science
# arts = WEATLists.W_8_Arts
# male = WEATLists.W_8_Male_terms
# female = WEATLists.W_8_Female_terms

white = WEATLists.W_5_Unused_full_list_European_American_names
black = WEATLists.W_5_Unused_full_list_African_American_names
pleasant = WEATLists.W_5_Pleasant
unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

All CN:  (128607, 300)
Without debiasing
WEAT d =  1.3542663
WEAT p =  0.0
(128607, 300)   (115, 300)
All CN:  (128607, 300)
With debiasing:  glove race_list
WEAT d =  0.73598015
WEAT p =  0.005
All CN:  (76078, 300)
Without debiasing
WEAT d =  -0.27154148
WEAT p =  0.263
(76078, 300)   (117, 300)
All CN:  (76078, 300)
With debiasing:  word2vec race_list
WEAT d =  -0.2778863
WEAT p =  0.255
All CN:  (119127, 300)
Without debiasing
WEAT d =  0.41335136
WEAT p =  0.039
(119127, 300)   (114, 300)
All CN:  (119127, 300)
With debiasing:  fasttext race_list
WEAT d =  0.21699712
WEAT p =  0.097
All mat:  (1161192, 768)
Number of words:  49815
All CN:  (1161192, 768)
Without debiasing
WEAT d =  1.3741117
WEAT p =  0.0
Subspace:  (751, 768)
(1161192, 768)   (751, 768)
Subspace mat:  (751, 768)
All CN:  (1161192, 768)
With debiasing:  elmo race_list
WEAT d =  1.0398372
WEAT p =  0.0


,Embedding,Subspace,Effect Size,p-value
0,glove,without_debiasing,1.3542663,0.0
1,glove,race_list,0.73598015,0.005
2,word2vec,without_debiasing,-0.27154148,0.263
3,word2vec,race_list,-0.2778863,0.255
4,fasttext,without_debiasing,0.41335136,0.039
5,fasttext,race_list,0.21699712,0.097
6,elmo,without_debiasing,1.3741117,0.0
7,elmo,race_list,1.0398372,0.0


## Bolukbasi hard debiasing

In [0]:
def doPCA(pairs, mat, index, num_components = 5):
    matrix = []
    for a, b in pairs:
        center = (mat[index[a.lower()]] + mat[index[b.lower()]])/2
        matrix.append(mat[index[a.lower()]] - center)
        matrix.append(mat[index[b.lower()]] - center)
    matrix = np.array(matrix)
    pca = PCA(n_components = num_components)
    pca.fit(matrix)
    # bar(range(num_components), pca.explained_variance_ratio_)
    return pca

def drop(u, v):
    return u - v * u.dot(v) / v.dot(v)
  
def normalize(all_words_mat):
    all_words_mat /= np.linalg.norm(all_words_mat, axis=1)[:, np.newaxis]
    return all_words_mat

In [0]:
def debias(all_words_mat, all_words_index, gender_specific_words, definitional, equalize):
    gender_direction = doPCA(definitional, all_words_mat, all_words_index).components_[0]
    specific_set = set(gender_specific_words)
    for w in list(all_words_index.keys()):
        if w not in specific_set:
            all_words_mat[all_words_index[w.lower()]] = drop(all_words_mat[all_words_index[w.lower()]], gender_direction)
    all_words_mat = normalize(all_words_mat)
    candidates = {x for e1, e2 in equalize for x in [(e1.lower(), e2.lower()),
                                                     (e1.title(), e2.title()),
                                                     (e1.upper(), e2.upper())]}
    print(candidates)
    for (a, b) in candidates:
        if (a.lower() in all_words_index and b.lower() in all_words_index):
            y = drop((all_words_mat[all_words_index[a.lower()]] + all_words_mat[all_words_index[b.lower()]]) / 2, gender_direction)
            z = np.sqrt(1 - np.linalg.norm(y)**2)
            if (all_words_mat[all_words_index[a.lower()]] - all_words_mat[all_words_index[b.lower()]]).dot(gender_direction) < 0:
                z = -z
            all_words_mat[all_words_index[a.lower()]] = z * gender_direction + y
            all_words_mat[all_words_index[b.lower()]] = -z * gender_direction + y
    all_words_mat = normalize(all_words_mat)
    return all_words_mat


In [137]:
import numpy as np
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

%cd debiaswe_tutorial/debiaswe/
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f) #gender definitional words
print("definitional", defs)
defs_list = []
for pair in defs:
  defs_list.append(pair[0])
  defs_list.append(pair[1])

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f) 
print("Equalize pairs", equalize_pairs)

%cd ../../
!ls

all_embd = ['glove', 'word2vec', 'fasttext']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
#all_subspace = ['without_debiasing', 'race_list']

math = WEATLists.W_7_Math
arts = WEATLists.W_7_Arts
male = WEATLists.W_7_Male_terms
female = WEATLists.W_7_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        #subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        #subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

/home/saketk/content/debiaswe_tutorial/debiaswe
definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
Equalize pairs [['monastery', 'convent'], ['spokesman', 'spokeswoman'], ['Catholic_priest', 'nun'], ['Dad', 'Mom'], ['Men', 'Women'], ['councilman', 'councilwoman'], ['grandpa', 'grandma'], ['grandsons', 'granddaughters'], ['prostate_cancer', 'ovarian_cancer'], ['testosterone', 'estrogen'], ['uncle', 'aunt'], ['wives', 'husbands'], ['Father', 'Mother'], ['Grandpa', 'Grandma'], ['He', 'She'], ['boy', 'girl'], ['boys', 'girls'], ['brother', 'sister'], ['brothers', 'sisters'], ['businessman', 'businesswoman'], ['chairman', 'chairwoman'], ['colt', 'filly'], ['congressman', 'congresswoman'], ['dad', 'mom'], ['dads', 'moms'], ['dudes', 'gals'], ['ex_girlfriend', 'ex_boyfriend'], ['father', 'mother'], ['fatherhood', 'motherhood'], ['fathers', 'moth

,Embedding,Subspace,Effect Size,p-value
0,glove,without_debiasing,1.0896145,0.019
1,glove,gender_list_extended,-0.60132134,0.863
2,glove,gender_list_propernouns,-0.57953775,0.864
3,glove,gender_list_pronouns,-0.5332187,0.845
4,glove,gender_list_all,-0.5322254,0.821
5,word2vec,without_debiasing,0.99810755,0.022
6,word2vec,gender_list_extended,-1.1440278,0.995
7,word2vec,gender_list_propernouns,-1.1701102,0.996
8,word2vec,gender_list_pronouns,-1.0880654,0.991
9,word2vec,gender_list_all,-1.0729237,0.988
